### **DAY 8 (16/01/26) – Unity Catalog Governance**

### **Feedback Form (Feel free to share your experience by feeling this form)**: https://forms.office.com/r/aZcUUPHUWf

### Learn:

- Catalog → Schema → Table hierarchy
- Access control (GRANT/REVOKE)
- Data lineage
- Managed vs external tables

### 🛠️ Tasks:

1. Create catalog & schemas
2. Register Delta tables
3. Set up permissions
4. Create views for controlled access

In [0]:
%sql

USE CATALOG workspace;


In [0]:
bronze=spark.read.format("delta").load("/Volumes/workspace/ecommerce/ecommerce_data/delta/bronze/events")
bronze.write.mode("overwrite").saveAsTable("bronze")

In [0]:
%sql
CREATE TABLE workspace.default.silver USING DELTA AS SELECT * FROM delta.`/Volumes/workspace/ecommerce/ecommerce_data/delta/silver/events`;

CREATE TABLE workspace.default.gold USING DELTA AS SELECT * FROM delta.`/Volumes/workspace/ecommerce/ecommerce_data/delta/gold/products`;

In [0]:
%sql
DROP CATALOG IF EXISTS ecommerce CASCADE;

In [0]:
%sql
CREATE CATALOG ecommerce;
USE CATALOG ecommerce;
CREATE SCHEMA bronze;
CREATE SCHEMA silver;
CREATE SCHEMA gold;

In [0]:
%sql
-- Register tables
CREATE TABLE ecommerce.bronze.events USING DELTA 
select * from delta.`/Volumes/workspace/ecommerce/ecommerce_data/delta/bronze/events`;

CREATE TABLE ecommerce.silver.events USING DELTA 
select * from delta.`/Volumes/workspace/ecommerce/ecommerce_data/delta/silver/events`;

CREATE TABLE ecommerce.gold.events USING DELTA 
select * from delta.`/Volumes/workspace/ecommerce/ecommerce_data/delta/gold/products`;


In [0]:
%sql
-- Permissions
GRANT SELECT ON TABLE ecommerce.gold.events TO `ashwinrajumapathy98@gmail.com`;
GRANT ALL PRIVILEGES ON SCHEMA ecommerce.silver TO `ashwinrajumapathy98@gmail.com`;

In [0]:
%sql
CREATE VIEW ecommerce.gold.top_products AS
SELECT brand, avg_revenue, conversion_rate
FROM ecommerce.gold.events
WHERE purchases > 10
ORDER BY avg_revenue DESC LIMIT 100;